In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

샘플로 제공된 video2.mp4를 moviepy로 읽어서 width=640으로 축소하여 화면에 플레이해 보고, 플레이한 내용을 mvpyresult.mp4라는 파일로 저장  
저장이 완료되면 샘플 원본과 새롭게 저장된 동영상 파일을 열어서 두 영상의 화면크기나 파일 사이즈를 비교

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4


moviepy 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장

In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 48/403 [00:00<00:00, 413.50it/s, now=None]

Moviepy - Building video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult2.mp4


3. 영상을 읽고 쓰는 시간을 측정. OpenCV를 사용할 때와 차이를 측정

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 48/403 [00:00<00:01, 334.69it/s, now=None]

Moviepy - Building video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/aiffel-dj17/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 3.40ms


In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.74ms


# moviepy 를 이용할 때의 장단점을 분석
- moviepy의 장점은 opencv보다 약 2배 빠르다는 것이다. 단점은 범용성이 적다. opencv는 python 말고도 c,c++, 자바, 매트랩, OCTAVE에서도 쓸수 있지만 moviepy는 파이썬에서만 사용가능하다.

## (4) 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성
- 문제 : 인원이 2명 이상일 때 제대로 작동하지 않는다
![ezgif com-gif-maker](https://user-images.githubusercontent.com/60597598/117774729-c2422f80-b274-11eb-85fb-9bf27da56b49.gif)
- mp4 : https://drive.google.com/file/d/1gwg36FzlDtkOf1Dhp8bzccI_u3RGrlc9/view?usp=sharing

## 칼만필터 적용하기
```
cd ~/aiffel/video_sticker_app && python webcam_sticker_kf.py
```

In [ ]:
import numpy as np
import cv2
import dlib

from kfaddsticker import img2sticker_kf

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    #vc = cv2.VideoCapture(0)
    vc = cv2.VideoCapture('./images/video2.mp4')
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.
     # writer 초기화
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vw = cv2.VideoWriter('./images/result123.mp4', fourcc, 30, (1280, 720)) 
    
    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    for i in range(vlen):
    #while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker_kf(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        vw.write(cv2.resize(img_result, (1280,720))) # save
        
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break
            
    vw.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

## 칼만필터 gif
- 용량이 커서 해상도를 줄이는 과정에 조금 깨진것 처럼 보입니다.
- 칼만필터 gif 원본 mp4 : https://drive.google.com/file/d/1ZSadtwi6jTPjd224-KUti1N3ONjgcFSC/view?usp=sharing

![result123 (1)](https://user-images.githubusercontent.com/60597598/117763973-b3548080-b266-11eb-92c4-16361fe09531.gif)

## 코드 수정 사항
- 왕관 잘릴 때 오류 나는 문제 해결을 위해 코드 추가

- Out bound 오류 해결 
```
if refined_x < 0: 
elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]: 
```
조건문으로 해결
![Screenshot from 2021-05-11 17-08-17](https://user-images.githubusercontent.com/60597598/117781370-8b234c80-b27b-11eb-9e9b-42204d5e6b8d.png)

- img2sticker 메소드 detector_hog 피라미드 수 0에서 1로 변경
![Screenshot from 2021-05-11 17-12-06](https://user-images.githubusercontent.com/60597598/117782690-d38f3a00-b27c-11eb-8edb-47162a036e9b.png)


## 루브릭
1. 웹캠을 통한 스티커앱을 실행하고 다각도로 문제점을 분석하여 보았다.
- 거리, 인원수, 각도 등 다양한 측면에서의 문제점과 기술적 대안을 정리하였다.  
- moviepy의 장점은 opencv보다 약 2배 빠르다는 것이다. 단점은 범용성이 적다. opencv는 python 말고도 c,c++, 자바, 매트랩, OCTAVE에서도 쓸수 있지만 moviepy는 파이썬에서만 사용가능하다.
---
2. 스티커앱 초기버전이 가진 예외사항들을 분석하여 수정하였다.
- 프레임 좌표범위 예외처리 관련 오류를 수정하였다.
  

---
3. 칼만 필터를 웹캠 스티커앱에 적용하여 스티커의 안정성을 높여 보았다.  
- 칼만 필터를 적용한 스티커앱을 작성하여 실행해 보고 안정성 여부를 확인하였다.